In [ ]:
import pandas as pd
import numpy as np

na_values = ['NO CLUE', 'N/A', '0']
# the request is not a dataframe but
# TextFileReader object for iteration
requests = pd.read_csv('311_data.csv',
                       chunksize=1000000,
                       na_values=na_values,
                       low_memory=False)

def chunk_data(requests):
    chunk_list = []  # append each chunk df here
    # Each chunk is in df format
    for chunk in requests:
        chunk_list.append(chunk)
        # concat the list into dataframe
    df_concat = pd.concat(chunk_list)

    return df_concat

def fix_zip_codes(zips):
    # Truncate everything to length 5
    zips = zips.str.slice(0, 5)

    # Set 00000 zip codes to nan
    zero_zips = zips == '00000'
    zips[zero_zips] = np.nan

    return zips


df_req = chunk_data(requests)
df_req['Incident Zip'] = fix_zip_codes(df_req['Incident Zip'])


In [ ]:
#print(dfObj.head())
#print(dfObj.columns) # print column names
df_req.rename(columns={"Agency Name": "AgencyName", "Incident Zip": "IncidentZip", "Complaint Type": "ComplaintType"})

# Rename column names
df_req.columns = df_req.columns.str.strip().str.lower().str.replace(' ', '_').str.replace('(', '').str.replace(')', '')

#print(dfObj.columns) # print column names

dfObj_agency = df_req.agency.unique() #Find unique elements in the column
dfObj_agency_name = df_req.agency_name.unique() #Find unique elements in the column
dfObj_incident_zip = df_req.incident_zip.unique() #Find unique elements in the column
dfObj_complaint_type = df_req.complaint_type.unique() #Find unique elements in the column

# print unique values in these columns:
print('agency ' + str(len(dfObj_agency)))
print('agency_name ' + str(len(dfObj_agency_name)))
print('incident_zip ' + str(len(dfObj_incident_zip)))
print('complaint_type ' + str(len(dfObj_complaint_type)))


#print(dfObj_incident_zip) #Review unique names for missing data or odd format
print(dfObj_agency) #Review unique names for missing data or odd format

In [ ]:
df_req.incident_zip.replace(regex=True,inplace=True,to_replace=r'\D',value=r'')
df_req.complaint_type.replace(regex=True,inplace=True,to_replace=r'[^a-zA-Z ]+',value=r'')

In [ ]:
df_req.complaint_type.unique()

In [ ]:
import sys
np.set_printoptions(threshold=sys.maxsize)
df_req.incident_zip.unique()

In [ ]:
# total complaints count
df_count = df_req.groupby(['complaint_type','incident_zip', 'agency']).size().reset_index(name="Total Count")
pd.set_option('display.max_rows', 50000)  # or 1000
df_count.head(50)

In [ ]:
import dask.dataframe as dd
filename = '311_data.csv'
df = dd.read_csv(filename, dtype='str')
df = df.rename(columns={c: c.replace(' ', '') for c in df.columns})

In [ ]:
df.head()

In [ ]:
df.IncidentZip.replace(regex=True,to_replace=r'\D',value=r'')
df.ComplaintType.replace(regex=True,to_replace=r'[^a-zA-Z ]+',value=r'')

In [ ]:
import pandas as pd
df_count = df.groupby(['ComplaintType','IncidentZip', 'Agency']).size().reset_index()
pd.set_option('display.max_rows', 50000)  # or 1000
df_count.head(50)

In [ ]:
df_count_zip = df.groupby(['ComplaintType','IncidentZip']).size().reset_index()
df_count_zip.head()

In [ ]:
by_zip = df_count_zip.groupby('IncidentZip', group_keys = False).sum().reset_index()
by_zip.head(50)

In [ ]:
import pandas as pd
import numpy as np

dfObj = pd.read_csv("Book1.csv")

# Rename column names
dfObj.columns = dfObj.columns.str.strip().str.lower().str.replace(' ', '_').str.replace('(', '').str.replace(')', '')


dfObj_incident_zip_unique = dfObj.incident_zip.unique() #Find unique elements in the column
dfObj_complaint_type_unique = dfObj.complaint_type.unique() #Find unique elements in the column


Finaldf = pd.DataFrame(dfObj_complaint_type_unique, columns=['Complaint_Type'])

for complaint_type_item in dfObj_complaint_type_unique:
    df_temp1 = dfObj.loc[(dfObj['complaint_type'] == complaint_type_item)] #Extract dataframe with a particular complaint type
    for zip_code_item in dfObj_incident_zip_unique:
        df_temp2 = dfObj.loc[(dfObj['incident_zip'] == zip_code_item)] #Extract dataframe with a particular zip code
        occurences= len(df_temp2.index) #Count number of rows for the data frame which has a particular ip code for a particular complaint type
        Finaldf[str(zip_code_item)] = ''
        Finaldf[str(zip_code_item)] = np.where(Finaldf['Complaint_Type'] == str(complaint_type_item), occurences, '')
        #Fill in the cell in the 'zip code' column for a given row of a 'complaint type'

export_csv = Finaldf.to_csv(r'Output.csv', index=None, header=True)